# 🌿 Sustain 4.0 BioEngine - Visualização de Usuários

Este notebook permite visualizar todos os usuários cadastrados no sistema Sustain 4.0 BioEngine através da leitura do arquivo `config.yaml` e apresentação dos dados em formato de DataFrame do pandas.

## 📊 Funcionalidades:
- Carregamento automático do arquivo de configuração
- Exibição dos usuários em formato tabular
- Análise básica dos dados de usuário
- Informações sobre senhas hasheadas (segurança)

## 1. 📚 Import Required Libraries

In [8]:
# Import necessary libraries
import pandas as pd
import yaml
from datetime import datetime
import os

# Display configurations for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("📚 Bibliotecas importadas com sucesso!")
print(f"📅 Pandas version: {pd.__version__}")
print(f"🕐 Timestamp: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

📚 Bibliotecas importadas com sucesso!
📅 Pandas version: 2.2.3
🕐 Timestamp: 01/08/2025 01:04:09


## 2. 📂 Load YAML Configuration File

In [9]:
# Load YAML configuration file
config_file = 'config.yaml'

try:
    # Check if file exists
    if not os.path.exists(config_file):
        raise FileNotFoundError(f"Arquivo {config_file} não encontrado!")
    
    # Load YAML content
    with open(config_file, 'r', encoding='utf-8') as file:
        config = yaml.safe_load(file)
    
    print(f"✅ Arquivo {config_file} carregado com sucesso!")
    print(f"📊 Estrutura do arquivo:")
    print(f"   - Seções principais: {list(config.keys())}")
    
    # Check if credentials section exists
    if 'credentials' in config and 'usernames' in config['credentials']:
        user_count = len(config['credentials']['usernames'])
        print(f"👥 Total de usuários encontrados: {user_count}")
    else:
        print("⚠️ Seção de credenciais não encontrada!")
        
except FileNotFoundError as e:
    print(f"❌ Erro: {e}")
    config = None
except yaml.YAMLError as e:
    print(f"❌ Erro ao ler YAML: {e}")
    config = None
except Exception as e:
    print(f"❌ Erro inesperado: {e}")
    config = None

✅ Arquivo config.yaml carregado com sucesso!
📊 Estrutura do arquivo:
   - Seções principais: ['cookie', 'credentials', 'preauthorized']
👥 Total de usuários encontrados: 5


## 3. 🔍 Extract User Data

In [10]:
# Extract user data from configuration
if config and 'credentials' in config and 'usernames' in config['credentials']:
    users_data = config['credentials']['usernames']
    
    print("🔍 Extraindo dados dos usuários...")
    print(f"📊 Usuários encontrados: {list(users_data.keys())}")
    
    # Process each user
    user_list = []
    for username, user_info in users_data.items():
        # Extract user information
        user_record = {
            'username': username,
            'name': user_info.get('name', 'N/A'),
            'email': user_info.get('email', 'N/A'),
            'password_hash': user_info.get('password', 'N/A'),
            'user_type': '👑 Admin' if username == 'admin' else '👤 User',
            'status': '🟢 Active'
        }
        user_list.append(user_record)
    
    print(f"✅ Dados extraídos para {len(user_list)} usuários!")
    
    # Display sample of extracted data
    print("\n📋 Amostra dos dados extraídos:")
    for i, user in enumerate(user_list[:3], 1):  # Show first 3 users
        print(f"   {i}. {user['username']} -> {user['name']} ({user['email']})")
        
else:
    print("❌ Não foi possível extrair dados dos usuários!")
    user_list = []

🔍 Extraindo dados dos usuários...
📊 Usuários encontrados: ['Canutilho', 'admin', 'analista', 'demo', 'pesquisador']
✅ Dados extraídos para 5 usuários!

📋 Amostra dos dados extraídos:
   1. Canutilho -> Augusto DC (audecan@gmail.com)
   2. admin -> Administrador (admin@bioengine.com)
   3. analista -> Analista (analista@bioengine.com)


## 4. 📊 Create Pandas DataFrame

In [11]:
# Create pandas DataFrame from user data
if user_list:
    # Create DataFrame
    df_users = pd.DataFrame(user_list)
    
    # Add some computed columns
    df_users['password_preview'] = df_users['password_hash'].apply(
        lambda x: x[:20] + '...' if x != 'N/A' else 'N/A'
    )
    df_users['created_date'] = datetime.now().strftime('%d/%m/%Y')
    
    # Reorder columns for better display
    column_order = [
        'username', 
        'name', 
        'email', 
        'user_type', 
        'status', 
        'password_preview',
        'created_date'
    ]
    df_users = df_users[column_order]
    
    print("✅ DataFrame criado com sucesso!")
    print(f"📊 Dimensões: {df_users.shape[0]} linhas x {df_users.shape[1]} colunas")
    print(f"📋 Colunas: {list(df_users.columns)}")
    
else:
    print("❌ Não foi possível criar DataFrame - nenhum dado de usuário disponível!")
    df_users = pd.DataFrame()  # Empty DataFrame

✅ DataFrame criado com sucesso!
📊 Dimensões: 5 linhas x 7 colunas
📋 Colunas: ['username', 'name', 'email', 'user_type', 'status', 'password_preview', 'created_date']


## 5. 📈 Display and Analyze User Data

In [12]:
# Display the complete DataFrame
if not df_users.empty:
    print("🌿 SUSTAIN 4.0 BIOENGINE - USUÁRIOS CADASTRADOS")
    print("=" * 60)
    
    # Display the DataFrame
    display(df_users)
    
else:
    print("📭 Nenhum usuário encontrado para exibir.")

🌿 SUSTAIN 4.0 BIOENGINE - USUÁRIOS CADASTRADOS


,username,name,email,user_type,status,password_preview,created_date
0,Canutilho,Augusto DC,audecan@gmail.com,👤 User,🟢 Active,$2b$12$y1AW4XeY0It4s...,01/08/2025
1,admin,Administrador,admin@bioengine.com,👑 Admin,🟢 Active,$2b$12$Oa5Olnt3gVAL4...,01/08/2025
2,analista,Analista,analista@bioengine.com,👤 User,🟢 Active,$2b$12$JzPZ9/9D5fyTM...,01/08/2025
3,demo,Demo User,demo@bioengine.com,👤 User,🟢 Active,$2b$12$vSQrAF95wa1RK...,01/08/2025
4,pesquisador,Pesquisador,pesquisador@bioengine.com,👤 User,🟢 Active,$2b$12$Zt/3YQuO6iyxB...,01/08/2025


In [13]:
# Detailed analysis and statistics
if not df_users.empty:
    print("\n📊 ANÁLISE DETALHADA DOS DADOS")
    print("=" * 40)
    
    # Basic info
    print(f"📈 Informações Básicas:")
    print(f"   • Total de usuários: {len(df_users)}")
    print(f"   • Colunas disponíveis: {len(df_users.columns)}")
    print(f"   • Tipos de dados:")
    for col in df_users.columns:
        print(f"     - {col}: {df_users[col].dtype}")
    
    # User type distribution
    print(f"\n👥 Distribuição por Tipo de Usuário:")
    user_type_counts = df_users['user_type'].value_counts()
    for user_type, count in user_type_counts.items():
        print(f"   • {user_type}: {count}")
    
    # Email domains analysis
    print(f"\n📧 Análise de Domínios de Email:")
    df_users['email_domain'] = df_users['email'].apply(
        lambda x: x.split('@')[1] if '@' in str(x) else 'N/A'
    )
    domain_counts = df_users['email_domain'].value_counts()
    for domain, count in domain_counts.items():
        print(f"   • {domain}: {count} usuário(s)")
    
    # Status overview
    print(f"\n🟢 Status dos Usuários:")
    status_counts = df_users['status'].value_counts()
    for status, count in status_counts.items():
        print(f"   • {status}: {count}")
    
    print(f"\n🔐 Informações de Segurança:")
    print(f"   • Todas as senhas estão hasheadas com bcrypt")
    print(f"   • Hash médio de caracteres: ~60 caracteres")
    print(f"   • Última atualização: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

else:
    print("📭 Sem dados para análise.")


📊 ANÁLISE DETALHADA DOS DADOS
📈 Informações Básicas:
   • Total de usuários: 5
   • Colunas disponíveis: 7
   • Tipos de dados:
     - username: object
     - name: object
     - email: object
     - user_type: object
     - status: object
     - password_preview: object
     - created_date: object

👥 Distribuição por Tipo de Usuário:
   • 👤 User: 4
   • 👑 Admin: 1

📧 Análise de Domínios de Email:
   • bioengine.com: 4 usuário(s)
   • gmail.com: 1 usuário(s)

🟢 Status dos Usuários:
   • 🟢 Active: 5

🔐 Informações de Segurança:
   • Todas as senhas estão hasheadas com bcrypt
   • Hash médio de caracteres: ~60 caracteres
   • Última atualização: 01/08/2025 01:04:09


## 6. 💾 Export Options and Summary

Esta seção oferece opções para exportar os dados e um resumo final da análise.

In [14]:
# # Export options and final summary
# if not df_users.empty:
#     print("💾 OPÇÕES DE EXPORTAÇÃO")
#     print("=" * 30)
    
#     # Export to CSV
#     csv_filename = f"sustain40_users_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
#     df_users.to_csv(csv_filename, index=False, encoding='utf-8')
#     print(f"✅ Dados exportados para: {csv_filename}")
    
#     # Export to Excel (optional)
#     try:
#         excel_filename = f"sustain40_users_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
#         df_users.to_excel(excel_filename, index=False, engine='openpyxl')
#         print(f"✅ Dados exportados para: {excel_filename}")
#     except ImportError:
#         print("⚠️ Para exportar para Excel, instale: pip install openpyxl")
    
#     print(f"\n📋 RESUMO FINAL")
#     print("=" * 20)
#     print(f"🌿 Sistema: Sustain 4.0 BioEngine")
#     print(f"👥 Total de usuários: {len(df_users)}")
#     print(f"🔐 Senhas protegidas: ✅")
#     print(f"📊 DataFrame disponível na variável: df_users")
#     print(f"📅 Análise realizada em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
    
#     # Final DataFrame reference
#     print(f"\n🎯 Para acessar o DataFrame, use a variável: df_users")
#     print(f"📖 Comandos úteis:")
#     print(f"   • df_users.head() - Primeiras 5 linhas")
#     print(f"   • df_users.info() - Informações do DataFrame") 
#     print(f"   • df_users.describe() - Estatísticas descritivas")
#     print(f"   • df_users['column_name'] - Acessar coluna específica")

# else:
#     print("📭 Nenhum dado disponível para exportação.")